# 变量

tf.Variable 表示可通过对其运行操作来改变其值的张量。与 tf.Tensor 对象不同，tf.Variable 存在于单个 session.run 调用的上下文之外。具体 op 允许您读取和修改此张量的值。这些修改在多个 tf.Session 之间是可见的，因此对于一个 tf.Variable，多个工作器可以看到相同的值。

## 创建变量

In [1]:
import tensorflow as tf

In [2]:
my_variable = tf.get_variable(
    name="my_variable",
    shape=[1,2,3],
    dtype=tf.int32,
    initializer=tf.zeros_initializer
)
print(my_variable)

<tf.Variable 'my_variable:0' shape=(1, 2, 3) dtype=int32_ref>


## 变量集合
上面说到了，变量是存放到Session之外的，可以被多个Session看到，为了能让访问到变量，Tensorflow 提供了变量集合来存放变量的命名列表。
TensorFLow中维护了一下几个变量集合

|集合名称  |集合内容  |使用场景  |
|:--------|:--------|:--------|
|tf.GraphKeys.GLOBAL_VARIABLES|可以在多台设备间共享的变量|持久化TensorFlow模型|
|tf.GraphKeys.LOCAL_VARIABLES|保存在本机上的变量|用于零时变量，计数器等|
|tf.GraphKeys.TRAINABLE_VARIABLES|TensorFlow 将计算其梯度的变量|模型训练|
|tf.GraphKeys.SUMMARIES|日志生成相关的张量|TensorFlow计算可视化|
|tf.GraphKeys.QUEUE_RUNNERS|处理输入的QueueRunner|输入处理|
|tf.GraphKeys.MOVING_AVERAGE_VARIABLES|所有计算了华东平均值的变量|计算滑动平均值|


In [3]:
# 在创建变量时，通过collections设置变量集合
my_local = tf.get_variable("my_local", shape=(),
                           collections=[tf.GraphKeys.LOCAL_VARIABLES])

non_trainable_var = tf.get_variable("non_trainable_var", shape=(),
                                   trainable=False)

# 创建自定义集合
tf.add_to_collection("my_col_name", my_local)

# 检索集合中的变量
tf.get_collection("my_col_name")

[<tf.Variable 'my_local:0' shape=() dtype=float32_ref>]

## 设置放置方式

In [4]:
#将变量放置到GPU上

with tf.device("/device:GPU:0"):
    gpu_var = tf.get_variable("gpu_var", [1])

## 初始化变量
变量必须经过初始化之后才能使用，使用keras等高级框架，可以帮助开发者自动初始化变量

要在训练开始前一次性初始化所有可训练变量，请调用 tf.global_variables_initializer()。此函数会返回一个操作，负责初始化 tf.GraphKeys.GLOBAL_VARIABLES 集合中的所有变量。运行此操作会初始化所有变量。


In [5]:
with tf.Session() as sess:
    #单独初始化本地集合变量
    sess.run(my_local.initializer) 
    #初始化 GLOBAL_VARIABLES 中的所有变量
    sess.run(tf.global_variables_initializer()) 

## 使用变量

* **变量赋值**： 使用 assgin、assign_add 方法
* **读取变量值**: 使用 tf.Variable.read_value()

In [6]:
with tf.Session() as sess:
    v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
    assignment = v.assign_add(1) # 0 + 1
    w = v + 1
    tf.global_variables_initializer().run()
    sess.run(assignment)
    print(assignment.eval())
    with tf.control_dependencies([assignment]):
        z = v.read_value()
        print("z: {}".format(z.eval()))

2.0
z: 3.0


## 共享变量
TensorFlow 支持两种共享变量的方式：
* 显式传递 tf.Variable 对象。
* 在 tf.variable_scope 对象内隐式封装 tf.Variable 对象。


In [8]:
def conv_relu(input, kernel_shape, bias_shape):
    """采用relu激活函数的卷积层
    """
    weights = tf.get_variable("weights", kernel_shape,
                              initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", bias_shape,
                             initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights, strides=[1,1,1,1], padding="SAME")
    return tf.nn.relu(conv + biases)

In [9]:
# 使用 reuse=True 触发变量共享
with tf.variable_scope("model"):
  output1 = conv_relu(input1)
with tf.variable_scope("model", reuse=True):
  output2 = conv_relu(input2)
# scope.reuse_variables() 以触发重用
with tf.variable_scope("model") as scope:
  output1 = my_image_filter(input1)
  scope.reuse_variables()
  output2 = my_image_filter(input2)

NameError: name 'my_image_filter' is not defined